In [1]:
from sqlalchemy import create_engine
from sqlalchemy import Column, Integer, String, Text, DateTime, Float, Boolean, ForeignKey, Table
from sqlalchemy.orm import relationship
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base

In [2]:
DB_URI = 'postgresql://postgres@localhost:5532/sqla'
engine = create_engine(DB_URI, echo=True)
Session = sessionmaker(bind=engine)
s = Session()

In [15]:
# /!\ DROP MODEL IN DB /!\
# Base.metadata.drop_all(bind=engine)

In [16]:
Base = declarative_base()

In [17]:
DB_URI = 'postgresql://postgres@localhost:5532/sqla'
engine = create_engine(DB_URI, echo=True)
Session = sessionmaker(bind=engine)
s = Session()

In [18]:
def populate(model, rows):
    s.add_all([model(**row) for row in rows])
    s.commit()

In [19]:
def create(model):
    s.add(model)
    s.commit()

In [20]:
"""
A basic Many to One Relationship
"""

class PlayerModel(Base):
    """Data model for players."""
    __tablename__ = "players"

    id = Column(Integer,
                primary_key=True,
                nullable=False)
    team_id = Column(Integer,
                     ForeignKey('teams.id'),
                     nullable=False)
    name = Column(String(100),
                  nullable=False)
    position = Column(String(100),
                      nullable=False)

    # Relationships
    team = relationship("TeamModel", backref="players")
    
    def __repr__(self):
        return '<Person model {}>'.format(self.id)


class TeamModel(Base):
    """Data model for people."""
    __tablename__ = "teams"

    id = Column(Integer,
                primary_key=True,
                nullable=False,
    )
    name = Column(String(100),
                  nullable=False)
    city = Column(String(100),
                  nullable=False)

    def __repr__(self):
        return '<Team model {}>'.format(self.id)

In [21]:
# PlayerModel.__table__.drop(engine)
# TeamModel.__table__.drop(engine)

In [22]:


association_table = Table('association', Base.metadata,
    Column('left_id', Integer, ForeignKey('left.id')),
    Column('right_id', String, ForeignKey('right.id'))
)


class Parent(Base):
    __tablename__ = 'left'
    id = Column(Integer, primary_key=True)
    name = Column(String, nullable=False)
    children = relationship(
        "Child",
        secondary=association_table,
        back_populates="parents")


class Child(Base):
    __tablename__ = 'right'
    id = Column(String, primary_key=True, )
    parents = relationship(
        "Parent",
        secondary=association_table,
        back_populates="children")

In [23]:
# association_table.drop(engine)
# Child.__table__.drop(engine)
# Parent.__table__.drop(engine)

In [24]:
# /!\ WRITE MODEL IN DB /!\
Base.metadata.create_all(engine)

2020-02-04 15:03:55,782 INFO sqlalchemy.engine.base.Engine select version()
2020-02-04 15:03:55,784 INFO sqlalchemy.engine.base.Engine {}
2020-02-04 15:03:55,790 INFO sqlalchemy.engine.base.Engine select current_schema()
2020-02-04 15:03:55,791 INFO sqlalchemy.engine.base.Engine {}
2020-02-04 15:03:55,797 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-02-04 15:03:55,799 INFO sqlalchemy.engine.base.Engine {}
2020-02-04 15:03:55,806 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-02-04 15:03:55,808 INFO sqlalchemy.engine.base.Engine {}
2020-02-04 15:03:55,822 INFO sqlalchemy.engine.base.Engine show standard_conforming_strings
2020-02-04 15:03:55,824 INFO sqlalchemy.engine.base.Engine {}
2020-02-04 15:03:55,867 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
20

In [25]:
TEAMS = [
    { 'name': 'PSG', 'city': 'Paris', 'id': 1},
    { 'name': 'OM', 'city': 'Marseille', 'id': 2},
    { 'name': 'OL', 'city': 'Lyon', 'id': 3},
    { 'name': 'LOSC', 'city': 'Lille', 'id': 4},
]
populate(TeamModel, TEAMS)

2020-02-04 15:03:56,153 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-02-04 15:03:56,155 INFO sqlalchemy.engine.base.Engine INSERT INTO teams (id, name, city) VALUES (%(id)s, %(name)s, %(city)s)
2020-02-04 15:03:56,156 INFO sqlalchemy.engine.base.Engine ({'city': 'Paris', 'id': 1, 'name': 'PSG'}, {'city': 'Marseille', 'id': 2, 'name': 'OM'}, {'city': 'Lyon', 'id': 3, 'name': 'OL'}, {'city': 'Lille', 'id': 4, 'name': 'LOSC'})
2020-02-04 15:03:56,184 INFO sqlalchemy.engine.base.Engine COMMIT


In [26]:
PLAYERS = [
    {'name':'Mbappe', 'position': '9', 'team_id': '1'},
    {'name':'Neymar', 'position': '7', 'team_id': '1'},
    {'name':'Diawara', 'position': '8', 'team_id': '3'},
    {'name':'Benedeto', 'position': '9', 'team_id': '2'},
    {'name':'Payet', 'position': '7', 'team_id': '2'},
    {'name':'Pepe', 'position': '10', 'team_id': '4'},
]
populate(PlayerModel, PLAYERS)

2020-02-04 15:03:56,573 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-02-04 15:03:56,575 INFO sqlalchemy.engine.base.Engine INSERT INTO players (team_id, name, position) VALUES (%(team_id)s, %(name)s, %(position)s) RETURNING players.id
2020-02-04 15:03:56,577 INFO sqlalchemy.engine.base.Engine {'position': '9', 'team_id': '1', 'name': 'Mbappe'}
2020-02-04 15:03:56,587 INFO sqlalchemy.engine.base.Engine INSERT INTO players (team_id, name, position) VALUES (%(team_id)s, %(name)s, %(position)s) RETURNING players.id
2020-02-04 15:03:56,588 INFO sqlalchemy.engine.base.Engine {'position': '7', 'team_id': '1', 'name': 'Neymar'}
2020-02-04 15:03:56,592 INFO sqlalchemy.engine.base.Engine INSERT INTO players (team_id, name, position) VALUES (%(team_id)s, %(name)s, %(position)s) RETURNING players.id
2020-02-04 15:03:56,594 INFO sqlalchemy.engine.base.Engine {'position': '8', 'team_id': '3', 'name': 'Diawara'}
2020-02-04 15:03:56,598 INFO sqlalchemy.engine.base.Engine INSERT INTO player

In [27]:
populate(Parent, [{'id':1, 'name': 'x'}, {'id':2, 'name': 'y'}])
populate(Child, [
    {'id': 'FR',}, {'id': 'ES'}, {'id': 'IT'},{'id': 'EN'}
])

2020-02-04 15:03:56,884 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-02-04 15:03:56,886 INFO sqlalchemy.engine.base.Engine INSERT INTO "left" (id, name) VALUES (%(id)s, %(name)s)
2020-02-04 15:03:56,888 INFO sqlalchemy.engine.base.Engine ({'id': 1, 'name': 'x'}, {'id': 2, 'name': 'y'})
2020-02-04 15:03:56,896 INFO sqlalchemy.engine.base.Engine COMMIT
2020-02-04 15:03:56,901 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-02-04 15:03:56,903 INFO sqlalchemy.engine.base.Engine INSERT INTO "right" (id) VALUES (%(id)s)
2020-02-04 15:03:56,904 INFO sqlalchemy.engine.base.Engine ({'id': 'FR'}, {'id': 'ES'}, {'id': 'IT'}, {'id': 'EN'})
2020-02-04 15:03:56,921 INFO sqlalchemy.engine.base.Engine COMMIT


In [28]:
p = s.query(Parent).filter_by(id=1).first()

2020-02-04 15:03:57,280 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-02-04 15:03:57,283 INFO sqlalchemy.engine.base.Engine SELECT "left".id AS left_id, "left".name AS left_name 
FROM "left" 
WHERE "left".id = %(id_1)s 
 LIMIT %(param_1)s
2020-02-04 15:03:57,285 INFO sqlalchemy.engine.base.Engine {'id_1': 1, 'param_1': 1}


In [29]:
p.children

2020-02-04 15:03:57,642 INFO sqlalchemy.engine.base.Engine SELECT "right".id AS right_id 
FROM "right", association 
WHERE %(param_1)s = association.left_id AND "right".id = association.right_id
2020-02-04 15:03:57,644 INFO sqlalchemy.engine.base.Engine {'param_1': 1}


[]

In [30]:
p.children.append(s.query(Child).filter_by(id='ES').first())

2020-02-04 15:03:58,025 INFO sqlalchemy.engine.base.Engine SELECT "right".id AS right_id 
FROM "right" 
WHERE "right".id = %(id_1)s 
 LIMIT %(param_1)s
2020-02-04 15:03:58,026 INFO sqlalchemy.engine.base.Engine {'id_1': 'ES', 'param_1': 1}


In [31]:
s.add(p)
s.commit()

2020-02-04 15:03:58,323 INFO sqlalchemy.engine.base.Engine INSERT INTO association (left_id, right_id) VALUES (%(left_id)s, %(right_id)s)
2020-02-04 15:03:58,325 INFO sqlalchemy.engine.base.Engine {'left_id': 1, 'right_id': u'ES'}
2020-02-04 15:03:58,330 INFO sqlalchemy.engine.base.Engine COMMIT
